In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [129]:
# Specify k
k = 6

In [130]:
# Import k-dependent data
folder = 'high_diversity'
sequence_counts = pd.read_csv(f'data/sequence_counts/{folder}/sequence_counts_k{k}.csv', index_col=0)
sequences = pd.read_csv(f'data/real_sequences/sequences_k{k}.csv', index_col=0)
clusters_w_seq = pd.read_csv(f'data/synapse_clusters/with_sequences/clusters_k{k}.csv', index_col=0)
display(sequence_counts.head())
display(sequences.head())
display(clusters_w_seq.head())

,sequence,count,diversity
0,"(864691135739717012, 864691136388590711, 86469...",1,6
1,"(864691135739717012, 864691136388590711, 86469...",1,4
2,"(864691136378754261, 864691136903065778, 86469...",1,6
3,"(864691136041046102, 864691136422832687, 86469...",1,5
4,"(864691134917384458, 864691135338086118, 86469...",1,6


,sequence_rank_0,sequence_rank_1,sequence_rank_2,sequence_rank_3,sequence_rank_4,sequence_rank_5
cluster_id,,,,,,
0,864691135739717012,864691136388590711,864691135415448506,864691136740412764,864691135684083895,864691135463697733
1,864691135739717012,864691136388590711,864691135415448506,864691136388590711,864691136388590711,864691135420483058
2,864691136378754261,864691136903065778,864691135772039931,864691136740406620,864691135730118585,864691135730537145
3,864691136041046102,864691136422832687,864691135526099547,864691136422832687,864691136486788370,864691135866412566
4,864691134917384458,864691135338086118,864691136723314173,864691135937724981,864691135012855446,864691136134450827


,synapse_id_0,synapse_id_1,synapse_id_2,synapse_id_3,synapse_id_4,synapse_id_5,density_score,post_pt_root_id,post_cell_type,centroid_x,centroid_y,centroid_z,sequence_rank_0,sequence_rank_1,sequence_rank_2,sequence_rank_3,sequence_rank_4,sequence_rank_5
cluster_id,,,,,,,,,,,,,,,,,,
0,941001,900422,906864,932560,932857,923889,0.397512,864691134884741370,4P,572.974667,691.796667,1052.113333,864691135739717012,864691136388590711,864691135415448506,864691136740412764,864691135684083895,864691135463697733
1,958004,979837,932857,932560,912217,906864,0.413748,864691134884741370,4P,577.322667,692.268000,1054.240000,864691135739717012,864691136388590711,864691135415448506,864691136388590711,864691136388590711,864691135420483058
2,674327,704519,662648,676483,709257,678862,0.384314,864691134884742906,6P-IT,488.466000,875.476000,936.273333,864691136378754261,864691136903065778,864691135772039931,864691136740406620,864691135730118585,864691135730537145
3,687488,689522,657710,670465,673155,692136,0.379345,864691134884742906,6P-IT,553.638667,1000.280667,1026.933333,864691136041046102,864691136422832687,864691135526099547,864691136422832687,864691136486788370,864691135866412566
4,685121,2241694,711425,686754,702105,676480,0.381143,864691134884743930,6P-IT,586.533333,758.324000,750.860000,864691134917384458,864691135338086118,864691136723314173,864691135937724981,864691135012855446,864691136134450827


In [131]:
repeated_sequences = sequence_counts.loc[sequence_counts['count'] > 1]
display(repeated_sequences.head())
print(repeated_sequences.shape)

,sequence,count,diversity
21687,"(864691135699524642, 864691135699524642, 86469...",2,5
55474,"(864691136577494036, 864691135584446194, 86469...",2,4
56570,"(864691136722356078, 864691136175499782, 86469...",2,4


(3, 3)


In [132]:
display(repeated_sequences['count'].describe())

count    3.0
mean     2.0
std      0.0
min      2.0
25%      2.0
50%      2.0
75%      2.0
max      2.0
Name: count, dtype: float64

In [133]:
sequences_tuples = sequences.apply(lambda x: tuple(x), axis=1)
display(sequences_tuples.head())

cluster_id
0    (864691135739717012, 864691136388590711, 86469...
1    (864691135739717012, 864691136388590711, 86469...
2    (864691136378754261, 864691136903065778, 86469...
3    (864691136041046102, 864691136422832687, 86469...
4    (864691134917384458, 864691135338086118, 86469...
dtype: object

In [134]:
def get_cluster_ids(all_sequences, sequence):
    clusters = all_sequences.loc[all_sequences.apply(lambda x: (tuple(sequence) == tuple(x)) or (tuple(sequence) == tuple(x[::-1])))]
    return clusters.index.values.astype(int)

In [135]:
repeated_sequences_ints = repeated_sequences['sequence'].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
display(repeated_sequences_ints.head())

21687    [864691135699524642, 864691135699524642, 86469...
55474    [864691136577494036, 864691135584446194, 86469...
56570    [864691136722356078, 864691136175499782, 86469...
Name: sequence, dtype: object

In [136]:
syn_id_keys = [f'synapse_id_{i}' for i in range(k)]
all_synapse_ids = clusters_w_seq[syn_id_keys]
overlaps = []
for i, sequence in enumerate(repeated_sequences_ints):
    cluster_ids = get_cluster_ids(sequences_tuples, sequence)
    cur_synapse_ids = all_synapse_ids.loc[cluster_ids]

    unique_synapse_ids = len(set(cur_synapse_ids.values.flatten()))
    total_synapse_ids = len(cur_synapse_ids.values.flatten())
    synapse_overlap = 1 - unique_synapse_ids / total_synapse_ids
    overlaps.append(synapse_overlap)
print(len(overlaps))

3


In [138]:
print(overlaps)

[0.33333333333333337, 0.41666666666666663, 0.33333333333333337]


In [137]:
print(len(np.array(overlaps)[np.array(overlaps) > 0.35]))

1
